# IRIS PROCESSING NOTEBOOK (IRIS with no screen)

## ReadMe : 
- This notebook is designed for the spreadsheet WITHOUT A SCREEN.
- A few guidelines about the snow pit spreadsheet :
    - IRIS version has to be specified as IRIS_# in the IRIS sheet of the pit spreadsheet
    - NoData is filled as 9999 in the spreadsheet when logging the .txt file IRIS provides
    - Field notes are to be logged in the dedicated section of the IRIS sheet, not in the table
<br/><br/>

Author : Paul Billecocq, GRIMP, Université de Sherbooke QC Canada

Contact : paul.billecocq@usherbrooke.ca

In [2]:
import iris_core as iris
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook

### 1. Input spreadsheet path

In [3]:
FILENAME = 'data/no_screen/RoundHill_20190422.xlsx' #fill in with your specific filename (it has to be placed in the same folder as the notebook)

# Read IRIS data from the IRIS sheet of the file
data = pd.read_excel(FILENAME, sheet_name="IRIS")

### 2. Processing

#### IRIS data retrieval, cleaning no data, and average reflectance voltages

In [4]:
# List cleaning function
def nanify_list(input_list):
    return [np.nan if x == 9999 else x for x in input_list]

# IRIS version retrieval
iris_version = data["Version"].dropna().tolist()[0]

# Calibration data retrieval
spectralon = data["Spectralon (%)"].dropna().tolist()
spectralon = np.array(spectralon,dtype=float)
calibration_one = nanify_list(data["Calibration Voltage 1 (V)"].dropna().tolist())
calibration_two = nanify_list(data["Calibration Voltage 2 (V)"].dropna().tolist())
calibration_three = nanify_list(data["Calibration Voltage 3 (V)"].dropna().tolist())
calibration_list = [calibration_one, calibration_two, calibration_three]
calibration_scans = np.array([calib for calib in calibration_list if len(calib) > 0 ])

# Calibration voltage averaging
calibration = np.nanmean(calibration_scans, axis=0)
calibration = np.array(calibration,dtype=float)

# Measures data retrieval
height = data["Height (cm)"].dropna().tolist()
scan_one = nanify_list(data["Scan 1 (V)"].dropna().tolist())
scan_two = nanify_list(data["Scan 2 (V)"].dropna().tolist())
scan_three = nanify_list(data["Scan 3 (V)"].dropna().tolist())

# Measures voltage averaging
scans = np.array([scan_one, scan_two, scan_three])
measures = np.nanmean(scans, axis=0)

#### Compute IRIS parameters

In [5]:
# Voltage to reflectance polynomial fit
polynom = iris.calibration_polynom_fit(spectralon, calibration)

In [6]:
# Compute reflectance
reflectance = iris.voltage_to_reflectance(measures, polynom)

In [7]:
# Compute SSA from reflectance
ssa = iris.reflectance_to_ssa(reflectance, iris_version)

In [8]:
# Compute optical radius from SSA
optical_radius = iris.ssa_to_optical_radius(ssa)

#### Processed data preview

In [9]:
measures_df = pd.DataFrame.from_dict({
    'height': height,
    'voltage': measures,
    'reflectance': reflectance,
    'ssa': ssa,
    'optical radius': optical_radius
})

measures_df

,height,voltage,reflectance,ssa,optical radius
0,295,0.315000,14.59,7.23,0.452643
1,290,0.626000,31.29,19.85,0.164867
2,285,0.318000,14.75,7.32,0.447078
3,280,0.724333,36.74,26.73,0.122432
4,275,0.686000,34.61,23.80,0.137505
5,270,0.616000,30.74,19.26,0.169917
6,265,0.654333,32.86,21.63,0.151300
7,260,0.674667,33.98,23.00,0.142287
8,255,0.353000,16.57,8.29,0.394766
9,250,0.282333,12.91,6.39,0.512145


### 3. Write processed data in the spreadsheet

In [10]:
# Spreadsheet initialization
workbook = load_workbook(FILENAME) 
iris_sheet = workbook["IRIS"]

for i in range(len(measures)):
    iris_sheet.cell(row = i+2, column = 10).value = reflectance[i]
    iris_sheet.cell(row = i+2, column = 11).value = ssa[i]
    iris_sheet.cell(row = i+2, column = 12).value = optical_radius[i]

# Write staged changes in the spreadsheet
workbook.save(FILENAME.split('.')[0] + '_IRIS_processed.xlsx')